使用google drive上傳資料

In [1]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
import os
import time

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model


Using TensorFlow backend.


讀取圖片

In [0]:
PATH = "C:\Users\user\Downloads\Python\AUAOI"
!ls "C:\Users\user\Downloads\Python\AUAOI"

#寫入資料夾路徑
#寫入資料夾目錄下的檔名
data_path = PATH + '/AOIdata_100'
data_dir_list = os.listdir(data_path)

#設定一個list
img_data_list=[]

#使用雙層for迴圈讀取每一筆圖片檔
for dataset in data_dir_list:
    img list = os.listdir(data_path+'/'+dataset)
    print ('Loaded the images of dataset-'+dataset)
    for img in img_list:
        img_path = data_path+'/'+dataset+'/'+img
    
    #讀取圖片
    #將讀取到的圖片轉成array
    #在陣列最前面增加一個維度
    #進行標準化
    #把標準化的值存入list
img = image.load_img(img_path)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
img_data_list.append(x)

img_data = np.array(img_data_list)

print (img_data.shape)  #(600, 1, 224, 224, 3)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)  #(1, 600, 224, 224, 3)
img_data=img_data[0]
print (img_data.shape)  #(600, 224, 224, 3)

In [0]:
# 給每張圖片標上Label
num_classes = 6
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:100]=0
labels[100:200]=1
labels[200:300]=2
labels[300:400]=3
labels[400:500]=4
labels[500:600]=5

# one-hot encoding
Y = np_utils.to_categorical(labels, num_classes)
#將圖片與Label對應
x,y = shuffle(img_data,Y, random_state=2)
# 切割資料分為訓練(train)跟測試(test)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)


In [0]:

#設定input shape
###  CODE HERE ###  (≈ 1 lines)
image_input = Input(shape=(224, 224, 3))
#取用VGG16模型 include_top為是否保留最上層三層的全連接層
#include_top=ture 默認輸入為(224,224,3) 須調整則須改成 False
###  CODE HERE ###  (≈ 1 lines)
model = VGG16(input_tensor=image_input, include_top=False,weights='imagenet')
#輸出模型層數概況
###  CODE HERE ###  (≈ 1 lines)
model.summary()

last_layer = model.get_layer('fc2').output
out = Dense(num_classes, activation='softmax', name='output')(last_layer)
custom_vgg_model = Model(image_input, out)
custom_vgg_model.summary()

for layer in custom_vgg_model.layers[:-1]:
	layer.trainable = False

#設定模型的損失函數 優化器 列出準確率
###  CODE HERE ###  (≈ 1 lines)
custom_vgg_model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

t=time.time()
#輸入訓練模型需要的參數
hist = custom_vgg_model.fit(X_train, y_train, batch_size=32, epcochs=10, verbose=1, validation_data=(X_test, y_test))
print('Training time: %s' % time.time())

#評估訓練完的模型對測試資料的準確度
(loss, accuracy)=custom_vgg_model.evaluate(X_test, y_test, batch_size=10, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

#預測測試資料
#印出預測出的Label值
### START CODE HERE ###  (≈ 5 lines)
prediction = custom_vgg_model.predict(X_test)
predict = np.argmax(prediction,axis=1)
ans = np.argmax(y_test,axis=1)
print(predict)
print(ans)
### END CODE HERE ###



輸出loss與acc圖像

In [0]:
#%%
# %matplotlib notebook
import matplotlib.pyplot as plt
# visualizing losses and accuracy
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']
xc=range(10)

plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])
plt.show()


小練習
讀取AOIdata_test中的60張圖片
並使用訓練好的模型進行預測